In [47]:
'''
2. 感知机相关；利用tensorflow等工具定义简单的几层网络（激活函数sigmoid），递归使用链式法则来实现反向传播。
'''
import pandas as pd
import jieba
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [48]:
#==1、读一个有两个分类的新闻数据集==
with open(r'cnews2.txt','r',encoding='utf-8') as fnews:
    newsLines = fnews.readlines()
#print(newsLines)
catergory = []
contentN = []
for newsItem in newsLines:
    catergory.append(newsItem.split('\t')[0])#\t以tab符切分
    contentN.append(newsItem.split('\t')[1])

dictNews = {'type':catergory,'content':contentN}
    
df = pd.DataFrame(dictNews)
df.head()

,type,content
0,体育,湖人新秀客场酒店醉酒闹事 警局一游错过黄蜂之战新浪体育讯北京时间4月26日消息，据雅虎体育报...
1,体育,魔鬼训练场爬出真奇兵 邓华德：他值得所有人尊敬华奥星空上海2月16日电(记者 李旭)“彭飞就...
2,体育,灰熊若黑八合情又合理！ 马刺天敌实力被大大低估新浪体育讯在刚刚结束的比赛中，西部第一马刺在客...
3,体育,皇帝关键球26%准度强于韦德 用他最后一投不是乱来新浪体育讯北京时间4月26日，再强大的球队...
4,体育,科比拒绝接受进一步检查 三方式治疗第五场一定上新浪体育讯北京时间4月26日消息(洛杉矶时间4...


In [49]:
#==2、分词、去停用词==
#取文章内容
#print(stopWordsList)
lines = df.content.values#.tolist()可以不用加，values取出就是list格式了
#print(lines)#看下格式，jieba切分的文本需要list格式
#开始分词
segList = []
for line in lines:
    contentSeg = jieba.lcut(line,cut_all=False)#精确模式
    #print(segment) 
    if len(contentSeg) > 1 and contentSeg !='\r\n':
        #【注意此处是关键，以回车符为界按文章分组存回，而不是整个分好词以后所有文章分词存做list的一个元素】       contentSeg.append(seg_list)
        #【为什么不按文章分？因为先分好了以后才能找到\n换行符】
        #wordsList = [wd for wd in seg_list if wd not in stopWordsList]
        segList.append(contentSeg)
#print("==分词：==\n",segList)

#组装成dataFrame看下分好词的数据
dfSeg = pd.DataFrame({'segList':segList})#DataFrame传参字典即可组装完成
print("\n\n==每篇文章初次分词 DataFrame数据表展示==")
dfSeg.head()




==每篇文章初次分词 DataFrame数据表展示==


,segList
0,"[湖人, 新秀, 客场, 酒店, 醉酒, 闹事, , 警局, 一游, 错过, 黄蜂, 之战..."
1,"[魔鬼, 训练场, 爬, 出, 真, 奇兵, , 邓华德, ：, 他, 值得, 所有人, ..."
2,"[灰熊, 若, 黑八, 合情, 又, 合理, ！, , 马刺, 天敌, 实力, 被, 大大..."
3,"[皇帝, 关键球, 26%, 准度, 强于, 韦德, , 用, 他, 最后, 一投, 不是..."
4,"[科比, 拒绝接受, 进一步, 检查, , 三, 方式, 治疗, 第五场, 一定, 上, ..."


In [50]:

#读停用词表
with open(r'cnews.vocab.txt','r',encoding='utf-8') as fStop:
    stopWords = fStop.readlines()
#print(stopWords)
stopWordsList = []
for sw in stopWords:
    stopWordsList.append(sw.split('\n')[0])

#去停用词
wordsClean = []
for seg in segList:
    segClean = []
    for word in seg:
        if len(word) >0 and word not in stopWordsList:
            segClean.append(word)
    wordsClean.append(segClean)
#print(wordsClean)

#组装成DataFrame格式看效果
dfSeg_clean = pd.DataFrame({'wordsClean':wordsClean})
print("===去停用词后===")
dfSeg_clean.head()    

===去停用词后===


,wordsClean
0,"[湖人, 新秀, 客场, 酒店, 醉酒, 闹事, 警局, 一游, 错过, 黄蜂, 之战, 新..."
1,"[魔鬼, 训练场, 奇兵, 邓华德, 值得, 所有人, 尊敬, 华奥, 星空, 上海, 16..."
2,"[灰熊, 黑八, 合情, 合理, 马刺, 天敌, 实力, 大大, 低估, 新浪, 体育讯, ..."
3,"[皇帝, 关键球, 26%, 准度, 强于, 韦德, 最后, 一投, 不是, 乱来, 新浪,..."
4,"[科比, 拒绝接受, 进一步, 检查, 方式, 治疗, 第五场, 一定, 新浪, 体育讯, ..."


In [51]:
# ==组装数据==
#把上面分好词的词袋与label（最初文章所属类别列）组装起来
dfDatas = pd.DataFrame({'features':wordsClean,'label':df['type']})
dfDatas.head()

,features,label
0,"[湖人, 新秀, 客场, 酒店, 醉酒, 闹事, 警局, 一游, 错过, 黄蜂, 之战, 新...",体育
1,"[魔鬼, 训练场, 奇兵, 邓华德, 值得, 所有人, 尊敬, 华奥, 星空, 上海, 16...",体育
2,"[灰熊, 黑八, 合情, 合理, 马刺, 天敌, 实力, 大大, 低估, 新浪, 体育讯, ...",体育
3,"[皇帝, 关键球, 26%, 准度, 强于, 韦德, 最后, 一投, 不是, 乱来, 新浪,...",体育
4,"[科比, 拒绝接受, 进一步, 检查, 方式, 治疗, 第五场, 一定, 新浪, 体育讯, ...",体育


In [18]:
#==3、label值向量化==
print(dfDatas['label'].unique())#['体育' '财经']
#label向量映射
labelDict = {'体育':1, '财经':2}
#映射到数据表
dfDatas['label'] = dfDatas['label'].map(labelDict)
dfDatas.head()

['体育' '财经']


,features,label
0,"[湖人, 新秀, 客场, 酒店, 醉酒, 闹事, 警局, 一游, 错过, 黄蜂, 之战, 新...",1
1,"[魔鬼, 训练场, 奇兵, 邓华德, 值得, 所有人, 尊敬, 华奥, 星空, 上海, 16...",1
2,"[灰熊, 黑八, 合情, 合理, 马刺, 天敌, 实力, 大大, 低估, 新浪, 体育讯, ...",1
3,"[皇帝, 关键球, 26%, 准度, 强于, 韦德, 最后, 一投, 不是, 乱来, 新浪,...",1
4,"[科比, 拒绝接受, 进一步, 检查, 方式, 治疗, 第五场, 一定, 新浪, 体育讯, ...",1


In [52]:
#==4、划分数据集==
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = \
    train_test_split(dfDatas['features'].values,
                     dfDatas['label'].values,
                     test_size=0.30,
                    random_state=0)
#打印看下训练集与测试集数据数量比，划分比例为30%
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
print()

(49,)
(22,)
(49,)
(22,)



In [53]:
#拼接X_train，使符合向量化函数需要的格式
X_train_new = []
for item in X_train:
    try:
        #空格拼接分词
        sentence = ' '.join(item)
        #文章存入新训练集列表
        X_train_new.append(sentence)
    except:
        print(item,word_index)#打印有问题的行、词序号
        
print("==拼接后：==\n",X_train_new[1])

==拼接后：==
 中欧 债基 获批 发行 谢潞锦 今年 基金 新品 根据 中欧 基金 管理 有限公司 传来 消息 公司 旗下 债券 型基金 中欧 稳健 收益 债券 证券 投资 基金 已经 获得 中国证监会 批准 近期 发行 了解 中欧 稳健 收益 基金 固定 收益 资产 基金 资产 比例 低于 80% 股票 权证 固定 收益 资产 基金 资产 比例 高于 20% 基金 力争 控制 风险 稳定 收益 基础 投资者 增加 回报 其中 风险 特征 明显 



In [26]:
#==5、特征向量化==
#向量化X_train_new
from sklearn.feature_extraction.text import CountVectorizer

#实例化向量构造器CountVectorizer
cv = CountVectorizer()

print(cv)#打印可看向量构造器全部参数
#向量构造器训练X_train_new得到向量化的X_train_new
cvFit = cv.fit(X_train_new)
X_train_cvFit = cv.fit_transform(X_train_new)

print(X_train_cvFit.toarray())

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [54]:
#==6、搭建神经网络骨架，初始化参数==
n_hidden_1 = 256#隐藏层1
n_hidden_2 = 128#隐藏层2
n_input = 784#输入层
n_classes = 2#输出层分类

X = tf.placeholder('float',[None,n_input])#特征,[]内为数据尺寸
y = tf.placeholder('float',[None,n_classes])#label

stddev = 0.1 #标准差
#定义三层神经网络的权重
#random_normal(第一层的输入，第一层的输出，即第二层神经元)
weights = {
    'w1':tf.Variable(tf.random_normal([n_input,n_hidden_1],stddev=stddev)),
    'w2':tf.Variable(tf.random_normal([n_hidden_1,n_hidden_2],stddev=stddev)),
    'out':tf.Variable(tf.random_normal([n_hidden_2,n_classes],stddev=stddev))   
}
#定义三层神经网络的偏移量
#b为各层静态数量即可
biases = {
    'b1':tf.Variable(tf.random_normal([n_hidden_1])),
    'b2':tf.Variable(tf.random_normal([n_hidden_2])),
    'out':tf.Variable(tf.random_normal([n_classes])),
}

In [58]:
#==7、前向传播函数（各层传递参数输入X计算输出二分类）
def perceptFn(_X,_weights,_biases):
    #传递第一层
    #公式：y = wx + b
    #sigmoid()激活函数，把线性输入输入映射到非线性的二分类概率值
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(_X,_weights['w1']),_biases['b1']))
    #传递第二层
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1,_weights['w2']),_biases['b2']))
    #传递第三层，输出到结果层
    layer_out = tf.nn.sigmoid(tf.matmul(layer_2,_weights['out'])+_biases['out'])
    
    return layer_out
    


In [64]:
#执行前向传播函数传参
pred = perceptFn(X,weights,biases)

#==8、反向传播，把预测值逐层传回，用梯度下降优化调整权重值==

#损失函数
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = pred,labels=y))
#sigmoid_cross_entropy交叉熵，用来算预测值与真实值差异
#reduce_mean求平均
#梯度下降使损失函数最小,实现最优控制
optm = tf.train.GradientDescentOptimizer(learning_rate = 0.01).minimize(cost)
#验证，比对预测值最大索引和真实值最大索引因同，则预测正确
corr = tf.equal(tf.argmax(pred,1),tf.argmax(y,1))
accr = tf.reduce_mean(tf.cast(corr,'float'))#将True False转1，0,求平均


In [ ]:
#==9、tensorflow处理执行以上函数==

#XTf = tf.placeholder('float',[None,None]) 

#初始化全局变量
init = tf.global_variables_initializer()#中词别忘加s
ephoches = 5 #迭代次数
batch_size = 10 #第组多少条数据
#初始化会话
sess = tf.Session()
sess.run(init)
#开始迭代
for epoch in range(ephoches):
    avg_cost = 0 #平均损失
    #数据分组
    oneBatch = int(X_train.shape[0]/batch_size)
    #遍历一次迭代中的一组数据batch
    for i in range(oneBatch):
        
        batch_X,batch_y= X_train.next_batch(batch_size)#next_batch向下执行batch
        #X,y placehold参数字典
        feeds ={X:batch_X,y:batch_y}
        #每迭代一次，传递本次X,y，进行一次梯度下降，优化拟合
        sess.run(optm,feed_dict = feeds)
        avg_cost += sess.run(cost,feed_dict=feeds)
        #下降后run 损失函数，传递本次X y，分数累加
    #求平均分 = 总分/一组batch总数量
    avg_cost = avg_cost/oneBatch
    
    #打印
    print("迭代第 %s 次，共 %s 次，平均分，%s.9f" %(epoch,epochs,avg_cost))
    feeds2 = {X:batch_x,y:batch_y}
    trainRight = sess.run(accr,feed_dict=feeds2)#执行预测函数，传递本次X，y
    print("预测准确率 %s.3f " % trainRight)   
    #测试集X，y赋值
    feedsTest = {X:X_test,y:y_test}#在划分数据集时y就是用y分的，分类label，不用再取label了
    testRight = sess.run(accr,feed_dict = feedsTest)
    print("测试准确率 %s.3f" % testRight)

#====》【X_train等数据需feed给tf，待调】

In [ ]:
'''
1. 前馈神经网络、网络层数、输入层、隐藏层、输出层、隐藏单元、激活函数的概念。

前馈神经网络：
是一种最简单的神经网络，各神经元分层排列。
每个神经元只与前一层的神经元相连。接收前一层的输出，并输出给下一层．各层间没有反馈。

网络层数:
有1个输入层，可以有多个隐藏层，1个输出层

输入层：
在输入层输入多个特征x，每个x和多个权重值w组合线性回归 wx+b 结果传递给下一层

隐藏层：
接收前一层的结果，经过激活函数映射为非线性概率值，并将结果继续向下一层专递

输出层：
接收最近一层隐藏层的分类概率值，经过线性回归公式wx+b 得到属于每个分类的得分

激活函数：
位于隐藏层的神经元，负责接收线性回归结果，转成非线性值，比如分类概率值。

2. 代码见上文

3. 激活函数的种类以及各自的提出背景、优缺点。（和线性模型对比，线性模型的局限性，去线性化）

sigmoid激活函数，早期用，它是逻辑回归函数，可以把输入的任意实数转成0到1之间的概率值。
sigmoid函数输入一个实值的数，然后将其压缩到0~1的范围内。
优点是能够很好的表达“激活”的意思，未激活就是0，完全饱和的激活则是1。
缺点是输入值很大或很小的时候sigmoid函数曲线趋近平行x轴，
在用梯度下降迭代最优模型时出现梯度消失，使训练不能再正常进行下去，
所以训练大量数据后逐渐出现问题，被Relu函数取代。

公式：f(z) = 1/1-e^z

Relu:线性整流函数，函数功能是比较输入值和0返回最大值，
画图表式为一条原点出发向斜右上方45延绅的直线
输入<0时，输出0；输入>0时,输入与输出相等。
优点是可以解决梯度消失问题，
容易求偏导，
收敛速度快
计算简单。
缺点：学习率很大时容易出现神经元坏死，
比如输入<0的值时，输入梯度为0，不会再有梯度，以后永远为0，也不会再被激活。
 
 公式：f(x) = max(0, x)
 
 
Leaky ReLU：为了解决负数导数被置0的情况，引入Leaky ReLU
当x<0时，给它的值加一个较小斜率偏导(如0.01等)的函数。
这样，既修正了数据分布，又保留了一些负轴的值，使得负轴信息不会全部丢失。

公式：f(x)=1(x<0)(ax)+1(x>=0)(x),其中a是一个很小的常数。


和线性模型对比，线性模型的局限性，去线性化：

线性模型比较简单也单一，只能划发简单分类，非线性比较灵活，能做复杂的神经网络分类识别等任务。
神经网络结合了线性和非线性的优点，输入端计算是线性回归，通过激活函数映射到非线性。

4. 深度学习中的正则化（参数范数惩罚：L1正则化、L2正则化；数据集增强；噪声添加；early stop；Dropout层）、正则化的介绍。

神经网络因为强大的灵活性，极易出现过拟合，使在训练集完美的模型在测试集表现不佳，

避免过拟合的措施：
加入正则惩罚项使模型更平稳，泛化能力更强，更有用。
正则征罚项是在拟合公式中加入一个固定的参数惩罚项，
L1正则惩罚项 加入一个绝对值，
L2正则惩罚项给w权重算个平方，使结果值大的更大，小的更小，拉开距离，更可分，泛化能力强。

神经元越多，越会过拟合：避免使用过多的w权重值可以防止过拟合
全连接过多也容易过拟合：策略是每次随机Drop out 一部分神经元（即w，一端连接神经元）

数据增强：使用更多的训练数据进行训练，必要时自己加一些假数据。

噪声添加：在神经网络中输入噪声（人为填加不相关的数据干扰）也可以看做是数据增强的一种方式.

提前终止:训练模型时,发现训练集的误差先减小再增大现象,可以提前终止算法减缓过拟合现象。

利用多种集成学习方法。

辅助分类节点:有各间层输出做为分类，以小权重加入到最终分类结果中。

Batch Normalization：对每一个mini-batch数据进行标准化(normalization)处理,使输出规范到N(0,1)的正太分布

5. 深度模型中的优化：参数初始化策略；自适应学习率算法（梯度下降、AdaGrad、RMSProp、Adam；优化算法的选择）；batch norm层（提出背景、解决什么问题、层在训练和测试阶段的计算公式）；layer norm层。

深度模型的优化原理：
前向传播通过随机初始化的w，计算出最终分类得分，
再把得分结果在反向传播中传回，通过梯度下降，迭代，使loss下降，不断的修正w。

参数初始化策略：权重随机初始化，标准化初使化。

自适应学习率算法：
每个参数设置不同的学习率，在整个学习过程中自动适应学习率

参考：
https://blog.csdn.net/huplion/article/details/79184338

BatchNorm：caffe框架中的常用层

Batchnorm是深度网络中经常用到的加速神经网络训练，加速收敛速度及稳定性的算法，可以说是目前深度网络必不可少的一部分。 
参考：https://blog.csdn.net/wfei101/article/details/79676491


layer_norm层：跟局部响应归一标准化有关，
参考：https://blog.csdn.net/qq_34650787/article/details/80461450
'''